In [1]:
import numpy as np
import pandas as pd
import json
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Finish')

Finish


In [2]:
CLIENT_ID='ACMTNDD43ZRV5FUNZSFHOKKWA1SLXURVGASRRZTZJ2CDYNHJ'
CLIENT_SECRET='ERYWYURWXMFADXACC1IRBWXSV4S2BEGT2DNXX2VJS45POEWW'
limit = 500 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
VERSION = '20190623'

In [3]:
url = 'https://it.wikipedia.org/wiki/Quartieri_di_Napoli'
page = urlopen(url).read().decode('utf-8')
soup = BeautifulSoup(page, 'html.parser')

wiki_table = soup.body.table.tbody

In [4]:
def get_cell(element):
    cells = element.find_all('td')
    row = []
    
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
        
    return row

print('get_cell Defined')

get_cell Defined


In [5]:
def get_row():    
    data = []  
    
    for tr in wiki_table.find_all('tr'):
        row = get_cell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    
    return data
print('get_row Defined')

get_row Defined


In [6]:
data = get_row()
columns = ['Quarters', 'District', 'Municipality']
df = pd.DataFrame(data,columns=columns)
df.head()

,Quarters,District,Municipality
0,Arenella,Circoscrizione 11 Arenella,Municipalità 05
1,Avvocata,Circoscrizione 08 Avvocata - Montecalvario - S...,Municipalità 02
2,Bagnoli,Circoscrizione 01 Bagnoli,Municipalità 10
3,Barra,Circoscrizione 19 Barra,Municipalità 06
4,Chiaia,Circoscrizione 05 Chiaia - Posillipo - San Fer...,Municipalità 01


In [7]:
links = wiki_table.find_all('a')
Quarters = []
for link in links:
    if link['href'].find("Municipalit%C3%A0") == -1:
        Quarters.append(link.get('title').replace('(Napoli)',''))
del Quarters[0:5]
Quarters

['Arenella ',
 'Avvocata ',
 'Bagnoli ',
 'Barra ',
 'Chiaia',
 'Chiaiano',
 'Fuorigrotta',
 'Marianella',
 'Mercato ',
 'Miano',
 'Montecalvario',
 'Pendino',
 'Pianura ',
 'Piscinola',
 'Poggioreale ',
 'Ponticelli ',
 'Porto ',
 'Posillipo',
 "San Carlo all'Arena",
 'San Ferdinando ',
 'San Giovanni a Teduccio',
 'San Giuseppe ',
 'San Lorenzo ',
 'San Pietro a Patierno',
 'Scampia',
 '1992',
 'Secondigliano',
 'Secondigliano',
 'Soccavo',
 'Stella ',
 'Vicaria',
 'Vomero',
 'Zona Industriale ']

In [8]:
df = pd.DataFrame()
df['Quarters'] = Quarters
df

,Quarters
0,Arenella
1,Avvocata
2,Bagnoli
3,Barra
4,Chiaia
5,Chiaiano
6,Fuorigrotta
7,Marianella
8,Mercato
9,Miano


In [9]:
def get_coords_local(localidad, output_as='center'):
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             localidad+',Napoli,Campania,Italy',
                             '&format=json&polygon=0')
    response = requests.get(url).json()[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output

In [10]:
df2 = df.copy()

latitudeCln = []
longitudeCln = []
for index, row in df2.iterrows():
    print(row[0])
    lat, long = get_coords_local(localidad=row[0], output_as='center')
    latitudeCln.append(lat)
    longitudeCln.append(long)

df2['Latitude'] = latitudeCln
df2['Longitude'] = longitudeCln

df2.shape

Arenella 
Avvocata 
Bagnoli 
Barra 
Chiaia
Chiaiano
Fuorigrotta
Marianella
Mercato 
Miano
Montecalvario
Pendino
Pianura 
Piscinola
Poggioreale 
Ponticelli 
Porto 
Posillipo
San Carlo all'Arena
San Ferdinando 
San Giovanni a Teduccio
San Giuseppe 
San Lorenzo 
San Pietro a Patierno
Scampia
1992
Secondigliano
Secondigliano
Soccavo
Stella 
Vicaria
Vomero
Zona Industriale 


(33, 3)

In [11]:
df2

,Quarters,Latitude,Longitude
0,Arenella,40.853741,14.225307
1,Avvocata,40.852078,14.242672
2,Bagnoli,40.814694,14.171709
3,Barra,40.839954,14.319866
4,Chiaia,40.835269,14.228268
5,Chiaiano,40.890758,14.218025
6,Fuorigrotta,40.828352,14.196400
7,Marianella,40.890968,14.231026
8,Mercato,40.848357,14.271930
9,Miano,40.887580,14.253696


In [12]:
address = 'Napoli, Italy'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The decimal coordinates of Napoli are {}, {}.'.format(latitude, longitude))

The decimal coordinates of Napoli are 40.8359336, 14.2487826.


In [13]:
# make a copy of the dataframe to get it simply back if needed
df3 = df2.copy()
df3

# create map of Naples using latitude and longitude values
map_naples = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, local in zip(df3['Latitude'], df3['Longitude'], df3['Quarters']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_naples)  
    
map_naples

In [14]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
          # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, limit)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Quarter', 
                  'Quarter Latitude', 
                  'Quarter Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [15]:
# Use category id 4bf58dd8d48988d1f8931735 to only get the b&b
naples_venues_bAndb = getNearbyVenues(names=df3['Quarters'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1f8931735')
naples_venues_bAndb.head()

,Quarter,Quarter Latitude,Quarter Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arenella,40.853741,14.225307,why not? - bed & breakfast,40.852638,14.229328,Bed & Breakfast
1,Arenella,40.853741,14.225307,WELCOME Bed & Breakfast,40.855000,14.236667,Bed & Breakfast
2,Arenella,40.853741,14.225307,la bibliotechina,40.849147,14.232397,Bed & Breakfast
3,Arenella,40.853741,14.225307,B&B Luca Giordano,40.846462,14.227963,Bed & Breakfast
4,Arenella,40.853741,14.225307,Napolinice b&b,40.853958,14.236923,Bed & Breakfast


In [16]:
naples_venues_bAndb.shape

(541, 7)

In [17]:
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Quarter'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [18]:
map_naples_bAndB = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(naples_venues_bAndb, 'red', map_naples_bAndB)
map_naples_bAndB

In [19]:
# Use category id 4bf58dd8d48988d12d941735 to only get the monuments
naples_venues_monuments = getNearbyVenues(names=df3['Quarters'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d12d941735')
naples_venues_monuments.head()

,Quarter,Quarter Latitude,Quarter Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Avvocata,40.852078,14.242672,Guglia dell'Immacolata,40.846982,14.251202,Monument / Landmark
1,Avvocata,40.852078,14.242672,Statua del Nilo,40.848723,14.256168,Monument / Landmark
2,Avvocata,40.852078,14.242672,Via Santa Maria Antesaecula 109 (Casa Di Totò),40.858753,14.252017,Monument / Landmark
3,Avvocata,40.852078,14.242672,palazzo Maddaloni,40.846649,14.249607,Monument / Landmark
4,Chiaia,40.835269,14.228268,Castello Aselmeyer,40.838937,14.235142,Monument / Landmark


In [20]:
naples_venues_monuments.shape

(44, 7)

In [21]:
map_naples_monuments = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(naples_venues_monuments, 'green', map_naples_monuments)
map_naples_monuments

In [22]:
# Use category id 4bf58dd8d48988d1fd931735 to only get the metro stations
naples_venues_metro = getNearbyVenues(names=df3['Quarters'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1fd931735')
naples_venues_metro.head()

,Quarter,Quarter Latitude,Quarter Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arenella,40.853741,14.225307,Montedonzelli,40.853784,14.225509,Metro Station
1,Arenella,40.853741,14.225307,Metro Montedonzelli (L1),40.851328,14.232675,Metro Station
2,Arenella,40.853741,14.225307,Metro Vanvitelli (L1),40.851316,14.232714,Metro Station
3,Arenella,40.853741,14.225307,Metro Montedonzelli (L1),40.855838,14.224418,Metro Station
4,Arenella,40.853741,14.225307,Metro Quattro Giornate (L1),40.846258,14.225410,Metro Station


In [23]:
map_naples_metro = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(naples_venues_metro, 'yellow', map_naples_metro)
map_naples_metro

In [24]:
# Use category id 4bf58dd8d48988d1ca941735 to only get the pizzerias
naples_venues_pizza = getNearbyVenues(names=df3['Quarters'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1ca941735')
naples_venues_pizza.head()

,Quarter,Quarter Latitude,Quarter Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arenella,40.853741,14.225307,Da Michele,40.852753,14.228935,Pizza Place
1,Arenella,40.853741,14.225307,May Freends,40.850609,14.227391,Pizza Place
2,Arenella,40.853741,14.225307,Pizzeria Gigante,40.851327,14.232686,Pizza Place
3,Arenella,40.853741,14.225307,Rocco E I Suoi Fratelli,40.859536,14.221510,Pizza Place
4,Arenella,40.853741,14.225307,Pizzeria Alba,40.850755,14.233727,Pizza Place


In [25]:
naples_venues_pizza.shape

(630, 7)

In [26]:
map_naples_pizza = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(naples_venues_pizza, 'gold', map_naples_pizza)
map_naples_pizza

In [27]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Quarter').count()
    
    for n in startDf['Quarter']:
        try:
            startDf.loc[startDf['Quarter'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Quarter'] == n,columnTitle] = 0

In [28]:
df_data = df3.copy()
df_data.rename(columns={'Quarters':'Quarter'}, inplace=True)
addColumn(df_data, 'B&B', naples_venues_bAndb)
addColumn(df_data, 'Monuments', naples_venues_monuments)
addColumn(df_data, 'Metro Station', naples_venues_metro)
addColumn(df_data, 'Pizza', naples_venues_pizza)
df_data

,Quarter,Latitude,Longitude,B&B,Monuments,Metro Station,Pizza
0,Arenella,40.853741,14.225307,10.0,0.0,8.0,21.0
1,Avvocata,40.852078,14.242672,50.0,4.0,12.0,38.0
2,Bagnoli,40.814694,14.171709,1.0,0.0,1.0,9.0
3,Barra,40.839954,14.319866,0.0,0.0,0.0,3.0
4,Chiaia,40.835269,14.228268,38.0,1.0,2.0,34.0
5,Chiaiano,40.890758,14.218025,0.0,0.0,2.0,2.0
6,Fuorigrotta,40.828352,14.196400,3.0,0.0,7.0,30.0
7,Marianella,40.890968,14.231026,0.0,0.0,5.0,3.0
8,Mercato,40.848357,14.271930,48.0,1.0,7.0,40.0
9,Miano,40.887580,14.253696,0.0,0.0,0.0,3.0


In [29]:
# negative weight, because Gennaro wants to open a B&B and thus wants to avoid concurrence as much as possible
weight_bAndb = -1

# positive weight, because monuments are most important
weight_monuments = 1.5

# positive weight, because metro stations are most important
weight_metro = 1

# positive weight because pizzerias are even most attractive
weight_pizza = 2

In [30]:
df_weighted = df_data[['Quarter']].copy()

In [31]:
df_weighted['Score'] = df_data['B&B'] * weight_bAndb + df_data['Monuments'] * weight_monuments + df_data['Metro Station'] * weight_metro + df_data['Pizza'] * weight_pizza
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted

,Quarter,Score
11,Pendino,72.0
10,Montecalvario,71.5
16,Porto,71.0
21,San Giuseppe,69.5
6,Fuorigrotta,64.0
25,1992,60.5
19,San Ferdinando,60.0
30,Vicaria,46.5
1,Avvocata,44.0
8,Mercato,40.5


In [32]:
map_nap_result = folium.Map(location=[latitude, longitude], zoom_start=15)

nap_win = df3[df3['Quarters'] == 'Pendino']

for lat, lng, local in zip(nap_win['Latitude'], nap_win['Longitude'], nap_win['Quarters']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_nap_result) 

addToMap(naples_venues_bAndb[naples_venues_bAndb['Quarter'] == 'Pendino'], 'red', map_nap_result)
addToMap(naples_venues_monuments[naples_venues_monuments['Quarter'] == 'Pendino'], 'green', map_nap_result)
addToMap(naples_venues_metro[naples_venues_metro['Quarter'] == 'Pendino'], 'gold', map_nap_result)
addToMap(naples_venues_pizza[naples_venues_pizza['Quarter'] == 'Pendino'], 'fuchsia', map_nap_result)

map_nap_result


In [33]:
### Pendino would be the ideal choice